In [ ]:
cd '../'

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import trange

from src.utils_freq import rgb2gray, dct, dct2, idct, idct2, batch_dct2, getDCTmatrix

from models import linear_model
from src.linear_analysis import train_linear_model

from collections import defaultdict
import ipdb
import copy

import time

# hyper params initalization
_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_printoptions(linewidth=200, edgeitems=5, precision=4)

In [ ]:
args = {"itr": 200001, # 8001 works with 0.2, 2001 works with 0.1
        "d": 3,
        "lr": 1e-2} 

w_log = torch.zeros(args["d"], args["itr"], 4)

w_tilde_star = torch.zeros(args['d'], 1)
w_tilde_star[0] = 5
w_tilde_star[1] = 10
w_tilde_star[2] = 0

sigma_tilde = torch.zeros(args['d'], 1)
sigma_tilde[0] = 0.1
sigma_tilde[1] = 0.05
sigma_tilde[2] = 0.

w_star = idct(w_tilde_star)
model = linear_model(args["d"]).to(_device)
w_init = torch.randn_like(model.linear.weight)
w_init[0,0] = 0.01
w_init[0,1] = -0.01
w_init[0,2] = 0.02

print("|A(0)|: {}\t |B(0)|: {}".format(np.sqrt(3)/3*sigma_tilde[0]**2*w_tilde_star[0],
                                      np.sqrt(2)/2*sigma_tilde[1]**2*w_tilde_star[1]))

w_init = idct(w_init.t()).t()
model.linear.weight = torch.nn.parameter.Parameter(w_init)

for i, opt_method in enumerate(['gd', 'signGD', 'adam', 'rmsp']):
    
    print(opt_method)
    t0 = time.time()
    model_copy = copy.deepcopy(model)
    model_copy.to(_device)
    

    if opt_method in ['gd','signGD']:
        opt = optim.SGD(model_copy.parameters(), lr = args["lr"]) 
    elif opt_method == 'adam':
        opt = optim.Adam(model_copy.parameters(), lr = args["lr"])
    elif opt_method == 'rmsp':
        opt = optim.RMSprop(model_copy.parameters(), lr = args["lr"])
        
    log = train_linear_model(args, w_tilde_star, sigma_tilde, model_copy, opt, opt_method == 'signGD', _device)
    
    torch.save(log["w"], './notebook/synthetic-'+opt_method+'.pt')
    t1 = time.time()
    print('total time: {0:.2f}s'.format(t1-t0))